In [60]:
# transform .simhess files to .h5 with cta-pipe for further analysis with DL1 data handler
# based on https://cta-observatory.github.io/ctapipe/tutorials/lst_analysis_bootcamp_2018.html
#import astropy.units as u
#from astropy.coordinates import SkyCoord, AltAz

import ctapipe
from ctapipe.containers import ImageParametersContainer
from ctapipe.io import EventSource

from ctapipe.calib import CameraCalibrator

from ctapipe.image import hillas_parameters, tailcuts_clean, morphology_parameters, leakage_parameters
from ctapipe.image import concentration_parameters

#from ctapipe.reco import HillasReconstructor


#from copy import deepcopy
#import tempfile
import os
import h5py
import time
from datetime import timedelta

In [61]:
type2name_dict = {'LST':'HESS-II', 'MST':'HESS-I'}
def rename_cameras(source, type2name_dict):
    '''
    rename HESS cameras in the source derived from .simhess
    '''
    for t in source.subarray.tel.values():
        tel_type = t.type
        tel = type2name_dict[tel_type]
        t.camera.camera_name = tel
        t.camera.geometry.camera_name = tel
        t.type = tel
        t.name =tel
        t.optics.name = tel
    return 0

def cam_index2cam_type(source):
    """
        dictionary that translates camera index into camera type
        necessary for correct output of geometry tables from simhess to cta-pipe .h5
    """
    res = {}
    table = source.subarray.to_table()
    for i, ind in enumerate(table['camera_index']):
        if ind not in res:
            res[ind] = table['camera_type'][i]
    return res

def rename_hess_cameras(filename, source, path='/configuration/instrument/telescope/camera'):
    '''
    rename HESS cameras in the output .h5 file
    (move the corresponding subtables)
    '''
    h5_obj = h5py.File(filename, 'a')
    h5t = h5_obj[path]
    ind2type_dict = cam_index2cam_type(source)
    for ind, ctype in ind2type_dict.items():
        for key in h5t.keys():
            if key.find('_%s' % ind) > -1:
                key_new = key.replace('_%s' % ind, '_%s' % ctype)
                if key_new not in h5t.keys():
                    h5t.move(key, key_new)
    h5_obj.flush()
    h5_obj.close()
    return 0

def get_fn_out(folder_out, fn0):
    return folder_out + fn0.replace('simhess.gz', 'h5')

In [65]:
mode = 'phase2d' # 'HPC_test'
if mode == 'HPC_test':
    folder_in = '/home/saturn/caph/mpp228/CTA_data/Heidelberg_HESS_examples/'
    folder_out = folder_in[:-1] + '_h5/'
    fn0 = 'gamma_55deg_0deg_run42329___phase2d2_desert-ws0-crab-alt35-off1.0-pos2_cone5.simhess.gz'
    #fn0 = 'gamma_20deg_180deg_run1202___phase2d_simon3_desert-all_tel-_mix_atm_new_trig_thresh-sims-ref0.79_off0.5.simhess.gz'
    #fn0 = 'gamma_20deg_0deg_run8608___phase2d_simon3_desert-all_tel-correction_scheme_case_A_off0.5.simhess.gz'
    fns_in = [fn0]
elif mode == 'phase2d':
    folder_in = '/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/\
phase2d/sim_telarray/phase2d/0.0deg/Data/'
    folder_out = folder_in[:-1] + '_h5/'
    fns_in = os.listdir(folder_in)
    replace = False
    
if not os.path.isdir(folder_out):
    os.makedirs(folder_out)

In [66]:
print(len(fns_in))

7554


In [ ]:
t0 = time.time()
ntot = 0
for k, fn_in0 in enumerate(fns_in):
    print('File %i out of %i' % (k + 1, len(fns_in)))
    fn_in = folder_in + fn_in0
    fn_out = get_fn_out(folder_out, fn_in0)
    if (not replace) and os.path.isfile(fn_out):
        os.remove(fn_in)
        continue
    if mode == 'HPC_test':
        source = EventSource(fn_in, max_events=100)
    else:
        source = EventSource(fn_in)
    rename_cameras(source, type2name_dict)
    calib = CameraCalibrator(subarray=source.subarray)
    
    with ctapipe.io.DataWriter(event_source=source, output_path=fn_out, 
                               #write_stereo_shower=True, 
                               write_index_tables=True, 
                               write_mono_shower=True,
                               overwrite=True, write_parameters=True, write_images=True) as write_data:
        for i, event in enumerate(source):            
            try:
                calib(event)
                for tel_id, tel_data in event.dl1.tel.items():
                    #print(tel_id)
                    tel = source.subarray.tel[tel_id]
                    geom = tel.camera.geometry
                    image = tel_data.image
                    mask = tailcuts_clean(geom, image)
                    tel_data.image_mask = mask.astype(bool)

                    event.dl1.tel[tel_id].parameters = ImageParametersContainer()
                    #tel_data.parameters = ImageParametersContainer()
                    hillas_c = hillas_parameters(geom[mask], image[mask])
                    event.dl1.tel[tel_id].parameters.hillas = hillas_c
                    event.dl1.tel[tel_id].parameters.leakage = leakage_parameters(geom, image, mask)
                    event.dl1.tel[tel_id].parameters.morphology = morphology_parameters(geom, mask) 
                    event.dl1.tel[tel_id].parameters.concentration = concentration_parameters(geom, image, hillas_c)

                    # placeholder for the moment
                    event.simulation.tel[tel_id].true_parameters = ImageParametersContainer()
                    #event.simulation.tel[tel_id].true_parameters.hillas = hillas_c

                write_data(event)
                ntot += 1
            except:
                pass
            if (i + 1) % 10 == 0 and mode == 'HPC_test':
                print('%i events' % (i + 1))
            elif (i + 1) % 100 == 0 and mode != 'HPC_test':
                td = timedelta(seconds=(time.time() - t0))
                print('%i events, %i tot events, time since start: %s' % (i + 1, ntot, td))

        rename_hess_cameras(fn_out, source, path='/configuration/instrument/telescope/camera')

File 1 out of 7554
100 events, 99 tot events, time since start: 0:00:07.191669
200 events, 199 tot events, time since start: 0:00:13.331270
300 events, 299 tot events, time since start: 0:00:19.469437
400 events, 399 tot events, time since start: 0:00:25.440540
500 events, 499 tot events, time since start: 0:00:31.572708
600 events, 599 tot events, time since start: 0:00:37.710495
700 events, 699 tot events, time since start: 0:00:43.778593
800 events, 799 tot events, time since start: 0:00:50.299216
900 events, 899 tot events, time since start: 0:00:56.517980
1000 events, 999 tot events, time since start: 0:01:02.288628
1100 events, 1099 tot events, time since start: 0:01:08.540271
File 2 out of 7554
100 events, 1236 tot events, time since start: 0:01:17.938476
200 events, 1336 tot events, time since start: 0:01:22.928039
300 events, 1435 tot events, time since start: 0:01:28.173696
400 events, 1535 tot events, time since start: 0:01:32.799821
500 events, 1635 tot events, time since s

200 events, 13095 tot events, time since start: 0:13:56.672734
300 events, 13195 tot events, time since start: 0:13:59.807868
400 events, 13295 tot events, time since start: 0:14:02.842640
500 events, 13395 tot events, time since start: 0:14:06.103614
600 events, 13495 tot events, time since start: 0:14:09.480942
700 events, 13595 tot events, time since start: 0:14:12.875764
800 events, 13695 tot events, time since start: 0:14:16.546844
900 events, 13795 tot events, time since start: 0:14:20.260854
1000 events, 13895 tot events, time since start: 0:14:23.265797
1100 events, 13995 tot events, time since start: 0:14:26.028230
1200 events, 14095 tot events, time since start: 0:14:29.268552
1300 events, 14195 tot events, time since start: 0:14:32.625622
1400 events, 14295 tot events, time since start: 0:14:35.892795
1500 events, 14395 tot events, time since start: 0:14:39.421040
1600 events, 14495 tot events, time since start: 0:14:42.877302
1700 events, 14595 tot events, time since start:

200 events, 26064 tot events, time since start: 0:24:54.924638
300 events, 26164 tot events, time since start: 0:25:03.683900
400 events, 26263 tot events, time since start: 0:25:12.748907
500 events, 26362 tot events, time since start: 0:25:21.649065
600 events, 26462 tot events, time since start: 0:25:30.934427
700 events, 26560 tot events, time since start: 0:25:39.150114
800 events, 26660 tot events, time since start: 0:25:46.894390
900 events, 26759 tot events, time since start: 0:25:55.474304
File 16 out of 7554
100 events, 26957 tot events, time since start: 0:26:13.437855
200 events, 27057 tot events, time since start: 0:26:20.128618
File 17 out of 7554
100 events, 27240 tot events, time since start: 0:26:36.906031
200 events, 27340 tot events, time since start: 0:26:44.801683
300 events, 27439 tot events, time since start: 0:26:54.193499
400 events, 27539 tot events, time since start: 0:27:02.852928
500 events, 27639 tot events, time since start: 0:27:12.068198
600 events, 277

2600 events, 39185 tot events, time since start: 0:34:41.297762
2700 events, 39285 tot events, time since start: 0:34:44.478927
2800 events, 39385 tot events, time since start: 0:34:47.652995
2900 events, 39485 tot events, time since start: 0:34:50.886268
3000 events, 39585 tot events, time since start: 0:34:54.218890
3100 events, 39685 tot events, time since start: 0:34:57.629821
3200 events, 39785 tot events, time since start: 0:35:00.890026
3300 events, 39885 tot events, time since start: 0:35:04.175132
3400 events, 39985 tot events, time since start: 0:35:07.742115
3500 events, 40085 tot events, time since start: 0:35:11.191813
3600 events, 40185 tot events, time since start: 0:35:14.682097
3700 events, 40285 tot events, time since start: 0:35:18.252861
3800 events, 40385 tot events, time since start: 0:35:22.133331
3900 events, 40485 tot events, time since start: 0:35:25.427543
4000 events, 40585 tot events, time since start: 0:35:28.439409
4100 events, 40685 tot events, time sinc

300 events, 52248 tot events, time since start: 0:45:47.688491
File 25 out of 7554
100 events, 52423 tot events, time since start: 0:45:59.998367
200 events, 52523 tot events, time since start: 0:46:05.701593
300 events, 52623 tot events, time since start: 0:46:11.315061
File 26 out of 7554
100 events, 52787 tot events, time since start: 0:46:22.810712
200 events, 52887 tot events, time since start: 0:46:30.759890
300 events, 52987 tot events, time since start: 0:46:38.284093
400 events, 53087 tot events, time since start: 0:46:46.240547
File 27 out of 7554
File 28 out of 7554
100 events, 53220 tot events, time since start: 0:46:59.615577
200 events, 53319 tot events, time since start: 0:47:08.325679
300 events, 53419 tot events, time since start: 0:47:17.561168
400 events, 53519 tot events, time since start: 0:47:26.165564
500 events, 53619 tot events, time since start: 0:47:35.558818
600 events, 53719 tot events, time since start: 0:47:45.124637
File 29 out of 7554
100 events, 53837 

...

600 events, 868908 tot events, time since start: 14:17:42.074508
File 614 out of 7554
100 events, 869097 tot events, time since start: 14:17:57.888680
200 events, 869197 tot events, time since start: 14:18:03.914138
300 events, 869297 tot events, time since start: 14:18:09.590191
400 events, 869397 tot events, time since start: 14:18:14.886217
500 events, 869497 tot events, time since start: 14:18:20.640456
600 events, 869597 tot events, time since start: 14:18:26.571119
700 events, 869697 tot events, time since start: 14:18:32.484583
800 events, 869796 tot events, time since start: 14:18:38.295362
900 events, 869896 tot events, time since start: 14:18:44.312015
1000 events, 869996 tot events, time since start: 14:18:49.902010
1100 events, 870096 tot events, time since start: 14:18:55.935135
1200 events, 870196 tot events, time since start: 14:19:01.951288
1300 events, 870296 tot events, time since start: 14:19:07.194788
1400 events, 870396 tot events, time since start: 14:19:12.884198

500 events, 881802 tot events, time since start: 14:31:49.547467
File 627 out of 7554
100 events, 881923 tot events, time since start: 14:32:02.469222
200 events, 882022 tot events, time since start: 14:32:11.597462
300 events, 882121 tot events, time since start: 14:32:19.443480
400 events, 882220 tot events, time since start: 14:32:26.498949
500 events, 882320 tot events, time since start: 14:32:34.408453
600 events, 882419 tot events, time since start: 14:32:44.170455
700 events, 882518 tot events, time since start: 14:32:53.067212
800 events, 882618 tot events, time since start: 14:33:02.211095
900 events, 882718 tot events, time since start: 14:33:10.853418
1000 events, 882818 tot events, time since start: 14:33:20.043994
1100 events, 882918 tot events, time since start: 14:33:28.491925
1200 events, 883018 tot events, time since start: 14:33:38.151152
1300 events, 883117 tot events, time since start: 14:33:47.494145
1400 events, 883217 tot events, time since start: 14:33:56.274176

600 events, 894427 tot events, time since start: 14:43:23.753299
700 events, 894527 tot events, time since start: 14:43:32.439331
800 events, 894627 tot events, time since start: 14:43:42.356453
900 events, 894727 tot events, time since start: 14:43:51.779657
File 635 out of 7554
100 events, 894853 tot events, time since start: 14:43:59.476452
200 events, 894953 tot events, time since start: 14:44:02.285007
300 events, 895053 tot events, time since start: 14:44:05.612035
400 events, 895153 tot events, time since start: 14:44:08.968853
500 events, 895253 tot events, time since start: 14:44:12.370148
600 events, 895353 tot events, time since start: 14:44:15.677499
700 events, 895453 tot events, time since start: 14:44:18.986330
800 events, 895553 tot events, time since start: 14:44:22.361338
900 events, 895653 tot events, time since start: 14:44:25.170854
1000 events, 895753 tot events, time since start: 14:44:28.239019
1100 events, 895853 tot events, time since start: 14:44:31.218341
12

1900 events, 906903 tot events, time since start: 14:52:11.288565
2000 events, 907003 tot events, time since start: 14:52:14.272537
2100 events, 907103 tot events, time since start: 14:52:17.403585
2200 events, 907203 tot events, time since start: 14:52:20.922975
2300 events, 907303 tot events, time since start: 14:52:24.075530
2400 events, 907403 tot events, time since start: 14:52:27.041484
2500 events, 907503 tot events, time since start: 14:52:30.259763
2600 events, 907603 tot events, time since start: 14:52:33.456683
2700 events, 907703 tot events, time since start: 14:52:36.578255
2800 events, 907803 tot events, time since start: 14:52:39.538886
2900 events, 907903 tot events, time since start: 14:52:42.526028
3000 events, 908002 tot events, time since start: 14:52:45.784447
3100 events, 908102 tot events, time since start: 14:52:49.098963
3200 events, 908202 tot events, time since start: 14:52:52.092673
3300 events, 908302 tot events, time since start: 14:52:55.331278
3400 event

1400 events, 919465 tot events, time since start: 15:01:43.557508
1500 events, 919565 tot events, time since start: 15:01:49.299148
1600 events, 919665 tot events, time since start: 15:01:54.985132
File 644 out of 7554
100 events, 919804 tot events, time since start: 15:02:08.158132
200 events, 919904 tot events, time since start: 15:02:16.336090
300 events, 920004 tot events, time since start: 15:02:24.227614


In [13]:
fn_out

'/home/saturn/caph/mpp228/CTA_data/Heidelberg_HESS_examples_h5/gamma_55deg_0deg_run42329___phase2d2_desert-ws0-crab-alt35-off1.0-pos2_cone5.h5'

In [8]:
with h5py.File(fn_out, 'r') as h5_test:
    h5t = h5_test['/configuration/instrument/telescope/camera']
    print('Keys should contain HESS-I and HESS-II:')
    print(h5t.keys())


Keys should contain HESS-I and HESS-II:
<KeysViewHDF5 ['geometry_HESS-I', 'geometry_HESS-I.__table_column_meta__', 'geometry_HESS-II', 'geometry_HESS-II.__table_column_meta__', 'readout_HESS-I', 'readout_HESS-I.__table_column_meta__', 'readout_HESS-II', 'readout_HESS-II.__table_column_meta__']>


In [54]:
hillas_parameters?

In [57]:
print(td)

0:06:40
